# wk2 Demo - Intro to Hadoop Streaming
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information`__

Last week you implemented your first MapReduce Algorithm using a bash script framework. We saw that adding a sorting component to our framework allowed us to write a more efficient reducer script and perform word counting in parallel. In this notebook, we'll introduce a new framework: Hadoop Streaming. Like before, you'll write mapper and reducer scripts in python then pass them to the framework which will stream over your input files, split them into chunks and sort to your specification. Although Hadoop Streaming is rarely used in production anymore it is the precursor to systems like Spark and as such is a useful way to illustrate key concepts in parallel computation. By the end of this live session you should be able to:
* __... describe__ the main components and default behavior of the Hadoop Streaming framework.
* __... write__ a Hadoop MapReduce job from scratch.
* __... access__ the Hadoop Streaming UI and use it in debugging your jobs.
* __... design__ Hadoop MapReduce implementations for simple tasks like counting and ordering.
* __... explain__ why sorting with multiple reducers requires some extra work (as opposed to sorting with a single reducer).

**Note**: Hadoop Streaming syntax is very particular. Make sure to test your python scripts before passing them to the Hadoop job and pay careful attention to the order in which Hadoop job parameters are specified.

### Notebook Set-Up

In [1]:
!hadoop version

Hadoop 2.6.0-cdh5.16.2
Subversion http://github.com/cloudera/hadoop -r 4f94d60caa4cbb9af0709a2fd96dc3861af9cf20
Compiled by jenkins on 2019-06-03T10:43Z
Compiled with protoc 2.5.0
From source with checksum 79b9b24a29c6358b53597c3b49575e37
This command was run using /usr/lib/hadoop/hadoop-common-2.6.0-cdh5.16.2.jar


## Hadoop Streaming Docs
https://hadoop.apache.org/docs/r2.6.0/hadoop-mapreduce-client/hadoop-mapreduce-client-core/HadoopStreaming.html   
Hadoop streaming is a utility that comes with the Hadoop distribution. The utility allows you to create and run Map/Reduce jobs with any executable or script as the mapper and/or the reducer.

In [2]:
# reloads
%reload_ext autoreload
%autoreload 2


For convenience, let's set a few global variables for paths you'll use frequently. __`NOTE:`__ _you may need to modify the jar file and HDFS (or local home) directory paths to match your environment. The paths below should work on the course Docker image. Refer to_ [this debugging FAQ](https://github.com/UCB-w261/main/blob/master/HelpfulResources/Hadoop/debugging-hadoop.md) _if you are unsure of the correct paths or encounter errors._

In [3]:
# path to java archive files for the hadoop streaming application on your machine
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"

In [4]:
# hdfs directory where  we'll store files for this assignment
HDFS_DIR = "/user/root/demo2"
!hdfs dfs -mkdir {HDFS_DIR}

In [5]:
# local directory where you've cloned the course repo 
# in docker this is the path where your clone is mounted -- ADJUST AS NEEDED
HOME_DIR = "/media/notebooks"

In [6]:
!pwd

/media/notebooks/LiveSessionMaterials/wk02Demo_IntroToHadoop/master


In [7]:
# get path to notebook
PWD = !pwd

In [8]:
PWD

['/media/notebooks/LiveSessionMaterials/wk02Demo_IntroToHadoop/master']

In [9]:
PWD = PWD[0]

In [10]:
PWD

'/media/notebooks/LiveSessionMaterials/wk02Demo_IntroToHadoop/master'

In [11]:
# <--- SOLUTION --->
# ... for instructors ...
HOME_DIR = "/media/notebooks/Instructors"  
# ... for students ...
HOME_DIR = "/media/notebooks" 

In [12]:
# store notebook environment path
from os import environ
PATH  = environ['PATH']
# NOTE: we can pass this variable to our Hadoop Jobs using -cmdenv PATH={PATH}
# This will ensure that, among other things, Hadoop uses the right python version.
# You should not *need* this until the very last question in part 1.

### Load the Data

In this notebook, we'll continue working with the  _Alice in Wonderland_ text file from HW1 and the test file we created for debugging. Run the following cell to confirm that you have access to these files and save their location to a global variable to use in your Hadoop Streaming jobs. 

In [13]:
# make a data subfolder - RUN THIS CELL AS IS
!mkdir -p data

In [14]:
# (Re)Download Alice Full text from Project Gutenberg - RUN THIS CELL AS IS 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
!curl "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  170k  100  170k    0     0   238k      0 --:--:-- --:--:-- --:--:--  238k


In [15]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Overwriting data/alice_test.txt


In [16]:
# save the paths - RUN THIS CELL AS IS (if Option 1 failed)
ALICE_TXT = PWD + "/data/alice.txt"
TEST_TXT = PWD + "/data/alice_test.txt"

In [17]:
# confirm the files are there - RUN THIS CELL AS IS
!echo "######### alice.txt #########"
!head -n 6 {ALICE_TXT}
!echo "######### alice_test.txt #########"
!cat {TEST_TXT}

######### alice.txt #########
﻿The Project Gutenberg EBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere in the United States and most
other parts of the world at no cost and with almost no restrictions
whatsoever.  You may copy it, give it away or re-use it under the terms of
the Project Gutenberg License included with this eBook or online at
######### alice_test.txt #########
This is a small test file. This file is for a test.
This small test file has two small lines.


# Content Overview: MapReduce Programming Paradigm
The week 2 reading from _Data Intensive Text Processing with Map Reduce_ by Lin and Dyer gave a high level overview of the key issues faced by parallel computation frameworks. It also introduced the Hadoop MapReduce framework. Lets start by briefly reviewing some of the key concepts from this week's async:

> __DISCUSSION QUESTIONS:__  
* What is MapReduce? How does it differ from Hadoop?  
* What are the main ideas of the functional programming paradigm and how does MapReduce exemplify these ideas?
* What is the basic data structure used in Hadoop MapReduce?
* What does 'data/code co-location' mean? How does this principle contribute to the efficiency of a distributed computation?
* What is a race condition in the context of parallel computation? Give an example.
* What kind of _synchronization_ does Hadoop MapReduce perform by default? 
* What aspect of the synchronization process is computationally costly?
* What does Hadoop sort by? What is Hadoop's default sort order?
* Throughout this course we'll emphasize the goal of writing 'stateless' implementations? What does that mean?

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS__
* What is MapReduce? How does it differ from Hadoop?
> MapReduce is a programming model for processing large data sets with a parallel, distributed algorithm on a cluster. It has two phases: 'map' which applies a transformation(or filter) to all elements of a collection, and 'reduce' which aggregates/folds the elements of a collection. The term "MapReduce" is also commonly used to refer to an execution framework built around this paradigm. Hadoop MapReduce is one such popular open-source framework.  

* What are the main ideas of the functional programming paradigm and how does MapReduce exemplify these ideas?
> Hadoop MapReduce is based on the functional programing paradigm. It is a declarative programming paradigm, which means programming is done with expressions. In functional code, the output value of a function depends only on the arguments that are input to the function. Key ideas in this paradigm include avoiding state changes/mutable data, and using higher-order functions (functions which take other functions as inputs. Map and reduce are examples of higher-order functions.  

* What is the basic data structure used in Hadoop MapReduce?
>Hadoop MapReduce processes data in key-value pairs.  


* What does 'data/code co-location' mean? How does this principle contribute to the efficiency of a distributed computation?
> Data/code "colocation" is a principle in distributed processing by which we send the code to the node where a chunk of data is stored and perform the computation on that node. This is efficient because it limits network traffic.  

* What is a race condition in the context of parallel computation? Give an example.
> A race condition happens when multiple execution entities (e.g. threads, processes, etc) are accessing or modifying the same resource at the same time, and the order of access can impact the result. For example, suppose two concurrent threads or processes A and B attempt to increment the same memory location. Without proper synchronization, chances are that the following sequence will occur: A reads, B reads, A writes, B writes - with the final value being incremented only once. To overcome this conditon, the global/count variable must be locked during the update; otherwise a thread can read the count while other threads are updating it, and the final accumulation will be less than the true value.

* What kind of _synchronization_ does Hadoop MapReduce perform by default? 
> Hadoop MapReduce performs synchronization using a barrier between the map and reduce phases. The reduce phase will not start until all mappers are complete. Then Hadoop reorganizes (a.k.a. 'shuffles') the mapper output which will be input to the reduce phase which will receive records sorted by key. 

* What aspect of the synchronization process is computationally costly?
> This 'shuffle' is computationally costly because of the sorting and network transfer involved.  

* What does Hadoop sort by? What is Hadoop's default sort order? 
> By default, hadoop sorts by key in alpha-numeric order. 

* Throughout this course we'll emphasize the goal of writing 'stateless' implementations? What does that mean?
> We don't want our implementations to depend on any shared mutable information. This is related to the idea of avoiding race conditions & the optimality of 'embarassingly parallel' implementations. Note that we also like to limit the use of memory given that we are likely using a cluster of cheap machines that may have limited space.... but that in-memory aggregation on individual nodes is not a violation of the principle of 'statelessness' (though it might seem like we are storing a 'state' in that scenario, the 'statefulness' that we want to avoid is any shared information that needs to be updated across multiple nodes.)

# Preview: Hadoop Streaming Syntax
A basic Hadoop MapReduce job consists of three components: a mapper script, a reducer script and a  line of code in which you pass these scripts to the Hadoop streaming application/framework. The mapper and reducer can be any executable that will read from `stdin` and write to `stdout`, including a bash executable like`/bin/cat` which simply passes the lines of your file unchanged, or python scripts like the ones you wrote in HW1. 

To run your hadoop streaming job, you'll need an HDFS filepath for the input data. We can use the following line of code to load a local file into HDFS:  

In [18]:
# put the alice test file into HDFS - RUN THIS CELL AS IS
!hdfs dfs -put {TEST_TXT} {HDFS_DIR}

__`TIP:`__ _Recall that we set the global variable_ `HDFS_DIR` _above to point to a directory called_ `demo2` _inside_ `/user/root/`, _you confirm that we've successfully loaded the data into HDFS using the following line. You can learn more about this command and others like it by reading the [Hadoop File System Shell Guide](https://hadoop.apache.org/docs/r2.7.5/hadoop-project-dist/hadoop-hdfs/HDFSCommands.html#dfs)._

In [19]:
# confirm the data was loaded - RUN THIS CELL AS IS
!hdfs dfs -ls {HDFS_DIR}

Found 1 items
-rw-r--r--   1 root supergroup         94 2021-01-16 18:54 /user/root/demo2/alice_test.txt


Ok, with that little bit of preparation, we can now run the most basic Hadoop Streaming Job.

>__`DISCUSSION QUESTIONS`__ (_before you run the next cell ..._)   
* Read the 6 lines below, what do each of them tell the framework to do?
* What to the forward slashes indicate?
* What do you expect to happen when we run this cell? (what should the output be? will it be printed to the console?)

In [20]:
# first hadoop streaming job - RUN THIS CELL AS IS
!hdfs dfs -rm -r {HDFS_DIR}/test-output

rm: `/user/root/demo2/test-output': No such file or directory


In [21]:
!hdfs dfs -rm -r {HDFS_DIR}/test-output
!hadoop jar {JAR_FILE} \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input {HDFS_DIR}/alice_test.txt \
  -output {HDFS_DIR}/test-output \
  -cmdenv PATH={PATH}

rm: `/user/root/demo2/test-output': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob7164655353546169933.jar tmpDir=null
21/01/16 18:54:34 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 18:54:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 18:54:35 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/16 18:54:35 INFO mapreduce.JobSubmitter: number of splits:2
21/01/16 18:54:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610822007109_0001
21/01/16 18:54:36 INFO impl.YarnClientImpl: Submitted application application_1610822007109_0001
21/01/16 18:54:36 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610822007109_0001/
21/01/16 18:54:36 INFO mapreduce.Job: Running job: job_1610822007109_0001
21/01/16 18:54:43 INFO mapreduce.Job: Job job_1610822007109_0001 running in uber mode : false

Note that running the cell above doesn't actually show us the results. That's because the results get written directly to the output directory in HDFS. You can view the results using an `hdfs` command. (__`Note:`__ `test-output` _is an HDFS directory that was created by the 5th line in the last cell, we could have named it anything we wanted_):

In [22]:
# view the contents of the result directory - RUN THIS CELL AS IS
!hdfs dfs -ls {HDFS_DIR}/test-output/

Found 2 items
-rw-r--r--   1 root supergroup          0 2021-01-16 18:54 /user/root/demo2/test-output/_SUCCESS
-rw-r--r--   1 root supergroup         96 2021-01-16 18:54 /user/root/demo2/test-output/part-00000


In [23]:
# view the results themselves - RUN THIS CELL AS IS
!hdfs dfs -cat {HDFS_DIR}/test-output/part-00000 | head

This is a small test file. This file is for a test.	
This small test file has two small lines.	


>__`DISCUSSION QUESTIONS`__ (_after running the Hadoop Job._)   
* Do the results match your expectations?
* Scan the Hadoop Job logging, what information stands out to you?
* When would it be a bad idea to print the full results of a job to the console? what could we do instead?
* What goes wrong if you try re-running the Hadoop Streaming job? Discuss two potential solutions to this problem.

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS__
* When discussing results point out the /bin/cat again... these are the 'unity' mapper and reducer... they just return the lines they receive.

* When discussing the loggings goal here is just to get students familiar with how this looks... they may notice information about time & memory usage, or the count of mappers and reducers. We'll get into the Hadoop UI explicilty a bit later but you may choose to point this out now.

* When would it be a bad idea to print the full results of a job to the console? what could we do instead?
> For large output this is a waste of space. For really large output this could cause your notebook to crash. HDFS has a head command.

* What goes wrong if you try re-running the Hadoop Streaming job? Discuss two potential solutions to this problem.
> Goal here is for students to start to be comfortable receiving an error message. In this case, the Hadoop Job won't run if the output directory you specify is not empty. Two solutions 1) rename the output directory when you re-run the command (this will duplicate results) OR better still 2) use a Hadoop command like the one below to empty the output directory each time you re-run a job.

In [24]:
# <--- SOLUTION --->
# clear the output directory before re-running a job. Eg.
!hdfs dfs -rm -r {HDFS_DIR}/test-output

Deleted /user/root/demo2/test-output


# Breakout 1: WordCount in Hadoop MapReduce
For most of the Hadoop jobs you write this week, you will use python scripts similar to those you wrote in HW1 to serve as your mapper and reducer. Since the Hadoop Streaming framework implements the principle of data/code co-location, we'll need to provide the paths to these python scripts so that Hadoop can ship them to the nodes where the code will get run. We do this by adding an additional flag to the Hadoop streaming command: the `-files` parameter. This will be the first of a number of additional flags that you can added to your Hadoop Streaming jobs to specify how the framework should handle your data. __`TIP:`__ _Hadoop can be very particular about the order in which you specify optional configuration fields. As a good debugging practice we recommend that you always maintain working code by starting with a basic job like the one we provided above and then testing the command as you add or modify parameters one by one_.

For your first breakout activity we've provided an example of a Hadoop MapReduce job that performs word counting on an input file of your choice. The mapper and reducer are python scripts provided at __`WordCount/mapper.py`__ and __`WordCount/reducer.py`__. The Hadoop Streaming command is in a cell below. As you read through the example and go on to write your own Hadoop MapReduce jobs you may want to refer to Michael Noll's [blogpost on writing an MapReduce job](http://www.michael-noll.com/tutorials/writing-an-hadoop-mapreduce-program-in-python/) and/or the [Hadoop Streaming documentation](https://hadoop.apache.org/docs/r2.7.5/hadoop-streaming/HadoopStreaming.html).

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (before breakout 1)__
* Breakouts can be awkward, you should jump in as quickly as possible you'll have just 10min for this one.

* Note: for a lot of these tasks you'll be reading & modifying the python scripts in folders inside the current directory... highly recommend Jupyter lab's split screen for this.

### Breakout 1 Tasks:
* __a) read scripts & docstrings:__ Read through **`WordCount/mapper.py`** and **`WordCount/reducer.py`** scripts and pay attention to the docstrings. Note that they (briefly) explain what the script does and the expected input/output record formats. [`HINT`: _docstrings are a way to record information to help your reader (future-self/collaborator/grader) quickly orient to a piece of code. They should describe_ __what__  (_not_ how) _is being done. For more information refer to the [PEP 8 Style Guide for Python](https://www.python.org/dev/peps/pep-0008/)_] The use of docstrings is recommended in all code that is written for this class.

* __b) discuss:__ What are the 'keys' and what are the 'values' in this MapReduce job? What delimiter separates them when we write to standard output? How will you expect Hadoop to sort the records emitted by the mapper script? Why is this order important given how the reducer script is written?


* __c) run provided code:__ Run the cells provided to make sure that your mapper and reducer scripts are executable, load the input files into HDFS, and clear the HDFS directory where the job will write its output. You will need to do these preparation steps for all future Hadoop MapReduce jobs.


* __d) unit test:__ A good habit when writing Hadoop streaming jobs is to test your mappers and reducers locally before passing them to a Hadoop streaming command. An easy way to do this is to pipe in a small line of text. We've provided the unix code to do so and added a unix sort to mimic Hadoop's default sorting. Run these cells to confirm that our mapper and reducer work properly. (Observe how the reducer doesn't work without the sort).


* __e) code:__ We've provided the code to run your Hadoop streaming command on the test file. Read through this command and be sure you understand each parameter that we're passing in, then run it and confirm that the output performs word counting correctly. Finally, modify the code provided to run the Hadoop MapReduce job on the _Alice and Wonderland_ text instead of the test file. Remember that the input path you pass to the Hadoop streaming command should be a location in HDFS not a local path. Take a look at the output and confirm you get the same count for 'alice' as in HW1. Food for thought: _does the sorting match what you expected?_ 

**`part c`** Prep for Hadoop Streaming Job

In [25]:
# part c - make sure the mapper and reducer are executable (RUN THIS CELL AS IS)
!chmod a+x WordCount/mapper.py
!chmod a+x WordCount/reducer.py

In [26]:
# part c - load the input files into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal {TEST_TXT} {HDFS_DIR}
!hdfs dfs -copyFromLocal {ALICE_TXT} {HDFS_DIR}

copyFromLocal: `/user/root/demo2/alice_test.txt': File exists


In [27]:
# part c - clear the output directory (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/wordcount-output
# NOTE: this directory won't exist unless you are re-running a job, that's fine.

rm: `/user/root/demo2/wordcount-output': No such file or directory


**`part d`** Unit test your scripts.

In [28]:
# part d - unit test mapper script
!echo "foo foo quux labs foo bar quux" | WordCount/mapper.py

foo	1
foo	1
quux	1
labs	1
foo	1
bar	1
quux	1


In [29]:
# part d - unit test reducer script
!echo -e "foo	1\nfoo	1\nquux	1\nlabs	1\nfoo	1\nbar	1\nquux	1" | WordCount/reducer.py

foo	2
quux	1
labs	1
foo	1
bar	1
quux	1


In [30]:
# part d - systems text mapper and reducer together
!echo "foo foo quux labs foo bar quux" | WordCount/mapper.py | WordCount/reducer.py

foo	2
quux	1
labs	1
foo	1
bar	1
quux	1


In [31]:
# part d - systems text mapper and reducer together with sort (RUN THIS CELL AS IS)
!echo "foo foo quux labs foo bar quux" | WordCount/mapper.py | sort -k1,1 | WordCount/reducer.py

bar	1
foo	3
labs	1
quux	2


**`part e`** Hadoop streaming command. __`NOTE:`__ _don't forget to clear the output directory before re-running this cell (see part b above)_

In [32]:
# part e - Hadoop streaming job (RUN THIS CELL AS IS FIRST, then make your modification)
!hadoop jar {JAR_FILE} \
  -files WordCount/reducer.py,WordCount/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/alice_test.txt \
  -output {HDFS_DIR}/wordcount-output \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob8937670275732726476.jar tmpDir=null
21/01/16 20:14:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:14:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:14:22 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/16 20:14:22 INFO mapreduce.JobSubmitter: number of splits:2
21/01/16 20:14:22 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610822007109_0002
21/01/16 20:14:23 INFO impl.YarnClientImpl: Submitted application application_1610822007109_0002
21/01/16 20:14:23 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610822007109_0002/
21/01/16 20:14:23 INFO mapreduce.Job: Running job: job_1610822007109_0002
21/01/16 20:14:29 INFO mapreduce.Job: Job job_1610822007109_0002 running in uber mode : false
21/01/16 20:14:29 INFO mapreduce.Job:  map 0% reduce 0%
21/01

In [34]:
# <--- SOLUTION --->
# part e - Hadoop streaming job (RUN THIS CELL AS IS FIRST, then make your modification)
!hdfs dfs -rm -r {HDFS_DIR}/wordcount-output
!hadoop jar {JAR_FILE} \
  -files WordCount/reducer.py,WordCount/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/alice.txt \
  -output {HDFS_DIR}/wordcount-output \
  -cmdenv PATH={PATH}

Deleted /user/root/demo2/wordcount-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob8720437918753814626.jar tmpDir=null
21/01/16 20:15:13 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:15:13 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:15:13 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/16 20:15:13 INFO mapreduce.JobSubmitter: number of splits:2
21/01/16 20:15:13 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610822007109_0003
21/01/16 20:15:14 INFO impl.YarnClientImpl: Submitted application application_1610822007109_0003
21/01/16 20:15:14 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610822007109_0003/
21/01/16 20:15:14 INFO mapreduce.Job: Running job: job_1610822007109_0003
21/01/16 20:15:20 INFO mapreduce.Job: Job job_1610822007109_0003 running in uber mode : false
21/01/16 20:15:20 I

In [35]:
# part e - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/wordcount-output/part-0000* > WordCount/results.txt
# NOTE: we would never do this for a really large output file! 
# (but it's convenient for illustration in this assignment)

In [36]:
# part e - view results (RUN THIS CELL AS IS)
!head WordCount/results.txt
# NOTE: these words and counts should match your results in HW1

a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2


In [37]:
# part e - check 'alice' count (RUN THIS CELL AS IS after running the job on the full file)
!grep 'alice' WordCount/results.txt
# EXPECTED OUTPUT: 404

alice	404


>__DISCUSSION QUESTIONS (after breakout 1)__
* What are the 'keys' and what are the 'values' in this MapReduce job? What delimiter separates them when we write to standard output? How will you expect Hadoop to sort the records emitted by the mapper script? Why is this order important given how the reducer script is written?
* What was the default sorting you saw? When do you think this sorting happens?
* Why go to the trouble of writing the reducer this way? why not just use a dictionary to count the words?

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (after breakout 1)__
* What are the 'keys' and what are the 'values' in this MapReduce job? What delimiter separates them when we write to standard output? How will you expect Hadoop to sort the records emitted by the mapper script? Why is this order important given how the reducer script is written?
> The keys are words the values are counts. The default delimiter in Hadoop streaming is a tab, however other delimiters (eg. comma, backslash, etc) could be specified. Hadoop will sort the records alphabetically by key (in our case, the word). This is important because the reducer will stream over the input records and emit the sum of consecutive counts (that belong to the same word). If the records were not alphabetized then the reducer might emit the same word twice without fully adding up its occurences.

* What was the default sorting you saw? When do you think this sorting happens?
> Sort by key, this happens between mapper & reducer (reducer needs it in order to work properly!)

* Why go to the trouble of writing the reducer this way? why not just use a dictionary to count the words?
> This goes back to the lesson from HW1... if we can't guarantee how much memory will be available in the nodes in our cluster of cheap comodity hardware, we want to try storing as little information as possible.

# Breakout 2: Uppercase and Lowercase Counts

What if we didn't care about individual word counts but rather wanted to know how many of the words in the _Alice_ file are upper and lower case? We could retrieve this information easily using a Hadoop streaming job with the same reducer script as in Section 2 (i.e. __`WordCount/reducer.py`__) but a slightly different mapper. In this question you'll design and write your own Hadoop streaming job to do just this.

> __ DISCUSSION QUESTION:__  
> * What should the keys be for this task? [`HINT:` _we'll need a key for each thing we want to count._]

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (before breakout 2)__
* What should the keys be for this task? 
> We only need two keys: 'upper' and 'lower' (or something equivalent).

* NOTE: by 'upper case' we mean the first letter is uppercase, not the whole word.

### Breakout 2 Tasks:
* __a) code:__ Complete the docstring and code in the __`UpperLower/mapper.py`__ to create a mapper that reads each input line, splits it into words and emits an appropriate key-value pair for each one. [`HINT:` _we're going to use this mapper in conjunction with the reducer from Breakout 1 so your key-value format should look very similar to the one in Breakout 1's mapper._]


* __b) unit test:__ Run the provided cells to make your new mapper executable and test that it works as you expect.


* __c) code:__ We've provided the start of a Hadoop streaming command. Fill in the missing parameters following the example provided in Section 2. Run your Hadoop job on the test file to confirm that it works. When you are happy with the results replace the test filepath with the real _Alice in Wonderland_ filepath and rerun the job. We'll compare results when we come back from breakouts.

* __d) discuss:__ Like our bash script HW1, Hadoop automatically splits up your records to be processed in parallel on separate mapper and reducer "nodes" (called "tasks" by Hadoop). Judging from the jobs you've run so far, what are the default number of 'map tasks' and 'reduce tasks' that Hadoop uses? Does this framework allow us to directly control the number of mappers and reducers? [__`HINTS`__: _to answer the first part of this question, look at the "Job Counters" section in the logging from your Hadoop job; for the second part of this question refer back to Lin & Dyer p24 at the very bottom_]

In [38]:
# part a - run this cell after completing your portion of the code
!chmod a+x UpperLower/mapper.py

In [39]:
# part b - unit test your new mapper (RUN THIS CELL AS IS)
!echo "Foo foo Quux Labs foo bar quux" | UpperLower/mapper.py

upper	1
lower	1
upper	1
upper	1
lower	1
lower	1
lower	1


In [40]:
# part b - systems test your new mapper with the reducer from question 2 (RUN THIS CELL AS IS)
!echo "Foo foo Quux Labs foo bar quux" | UpperLower/mapper.py | sort -k1,1 | WordCount/reducer.py

lower	4
upper	3


In [41]:
# part c - clear output directory before (re)running your Hadoop Job (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/upperlower-output

rm: `/user/root/demo2/upperlower-output': No such file or directory


In [ ]:
# part c - Hadoop streaming command (FILL IN MISSING ARGUMENTS)
!hadoop jar {JAR_FILE} \
  -files UpperLower/mapper.py,WordCount/reducer.py \

    
    
  -output {HDFS_DIR}/upperlower-output \ 
  -cmdenv PATH={PATH}

In [42]:
# <--- SOLUTION --->
# part c - Hadoop streaming command (FILL IN MISSING ARGUMENTS)  
!hadoop jar {JAR_FILE} \
  -files UpperLower/mapper.py,WordCount/reducer.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/alice.txt \
  -output {HDFS_DIR}/upperlower-output \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob2504415732073681552.jar tmpDir=null
21/01/16 20:16:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:16:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:16:16 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/16 20:16:16 INFO mapreduce.JobSubmitter: number of splits:2
21/01/16 20:16:16 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610822007109_0004
21/01/16 20:16:16 INFO impl.YarnClientImpl: Submitted application application_1610822007109_0004
21/01/16 20:16:16 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610822007109_0004/
21/01/16 20:16:16 INFO mapreduce.Job: Running job: job_1610822007109_0004
21/01/16 20:16:23 INFO mapreduce.Job: Job job_1610822007109_0004 running in uber mode : false
21/01/16 20:16:23 INFO mapreduce.Job:  map 0% reduce 0%
21/01

In [43]:
# part c - results (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/upperlower-output/part-000* > UpperLower/results.txt
!cat UpperLower/results.txt

lower	25071
upper	3011


> __DISCUSSION QUESTIONS (after breakout 2)__
* How many uppercase/lowercase words did you find?
* How many map tasks? how many reduce tasks? Where did you find this information?

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (after breakout 2)__
* How many uppercase/lowercase words did you find?
> `lower	25071`
`upper	3011`

* How many map tasks? how many reduce tasks? Where did you find this information?
> By default Hadoop uses (at least) 2 mappers and 1 reducer. We can specify the number of reducers it should use and suggest a number of mappers, but we can't force it to use the number of mappers we suggest. Hadoop will make a determination based on the data size and location (and block size in HDFS).

# Breakout 3: Number of Unique Words

Another variation on the simple word counting job would be to count the number of unique words in the book (instead of counting the unique occurrences of each word). Of course in reality the easiest way to get this information would be to count the number of lines in our word count output file... but since our goal here is to practice designing and writing Hadoop jobs, let's pretend for a moment that we don't have access to that file and instead think about how we'd do this from scratch. In this question we'll also introduce an important flag you can add to your Hadoop streaming jobs to control the degree of parallelization. 

> __DISCUSSION QUESTIONS:__  
* What should our keys be for this task? 
* Does it make most sense to check for 'uniqueness' inside the mapper or inside the reducer? why? [`HINT:` _think about our discussion of memory constraints in HW1 and about the synchronization that Hadoop performs for us automatically between the map and reduce phases._]

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (before breakout 3)__
* what should the keys/values be?
> The keys should be words --> we need to sort these inorder to remove duplicates (& get the unique count)   

* where should we check 'uniqueness'?
> In the reducer after we've sorted (if we do it in mappers we could end up with duplicates).  Filtering for unique words inside the mapper would require maintaining a list of 'words seen' which could be memory intensive. Even if we decided to do this local aggregation, there could be duplicate keys emitted by map tasks on different nodes which means we can't fully filter out non-unique words in the map phase alone (unless we were to guarantee a single map task). Since the shuffle phase automatically groups records for each key together it will be easy and effcient to handle deduplication in the reducer.

### Breakout 3 Tasks:

* __a) code + unit test:__ Since the mapper we wrote for `WordCount` already emits the right keys, let's simply reuse that mapper. Fill in the missing code in __`VocabSize/reducer.py`__so that this new reducer processes the records emitted by __`WordCount/mapper.py`__ and outputs the count of the number of unique words that appear in the input file. Run the provided unit test to confirm that your reducer works as you want it to.


* __b) code:__ Write and run a Hadoop streaming job to calculate the number of unique words in _Alice and Wonderland_ and record it in the space provided (`NOTE:` _for 'c' you'll modify this job and overwrite the original result which is why we'll ask you to record it in markdown._)


* __c) code + discussion:__ Add the flag `-numReduceTasks 3` to the very end of the Hadoop streaming command you wrote for `part c`. This flag tells Hadoop to use 3 separate reduce tasks, in other words, we'll make 3 'partitions' from the records emitted by your map phase and perform the reducing on each part. Rerun the job with this added flag and observe the result.  
    * What do you notice about the contents of the HDFS output directory and the final output itself? How would we have to post process our results to get the answer we're looking for?


* __d) Hadoop UI:__ In addition to the logging that Hadoop prints to your notebook you can also access two UIs with more detailed information about your Hadoop streaming jobs. While your job is currently running you can track its progress on port `8088` (this will be especially helpful in latter assignments when we run jobs that may take a long time).The link to this 'Running Job Tracker' UI can be found near the top of the logging from your job. Look for a line that reads something like:
>`The url to track the job: http://quickstart.cloudera:8088/proxy/application_########_#####/`

 Once the job has completed, this link will redirect to the 'MapReduce Job History UI' on port `19888`. This is where you can access information about completed jobs (note the Job ID number will match the one printed in the URL above). 
  > `localhost:19888/jobhistory/job/job_########_#####`

 For `part d` Navigate to the MapReduce Job History UI (the one on port `19888`) and confirm that your job used 2 map tasks and 3 reduce tasks.  

In [44]:
# part b - write your code in the provided script first, then RUN THIS CELL AS IS
!chmod a+x VocabSize/reducer.py

In [45]:
# part b - unit test your new reducer (RUN THIS CELL AS IS)
!echo -e "foo	1\nfoo	1\nquux	1\nlabs	1\nfoo	1\nbar	1\nquux	1" | sort -k1,1 | VocabSize/reducer.py

NumUniqueWords	4


In [46]:
# part c - clear output directory before (re)running your Hadoop Job (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/vocabsize-output

rm: `/user/root/demo2/vocabsize-output': No such file or directory


 __`TIPS:`__ _When writing your job below make sure that you have the correct paths to your input file, output directory and mapper/reducer script. Don't forget the `-files` option, and_ DO NOT _put spaces between the paths that you pass to this option_.

In [ ]:
# parts b/c - write/modify your Hadoop streaming command here:


In [50]:
# <--- SOLUTION --->
# parts b/c - write/modify your Hadoop streaming command here:
!hdfs dfs -rm -r {HDFS_DIR}/vocabsize-output
!hadoop jar {JAR_FILE} \
  -files WordCount/mapper.py,VocabSize/reducer.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/alice.txt \
  -output {HDFS_DIR}/vocabsize-output \
  -numReduceTasks 3 \
  -cmdenv PATH={PATH}

Deleted /user/root/demo2/vocabsize-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob4401094221774663116.jar tmpDir=null
21/01/16 20:20:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:20:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:20:20 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/16 20:20:20 INFO mapreduce.JobSubmitter: number of splits:2
21/01/16 20:20:20 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610822007109_0006
21/01/16 20:20:20 INFO impl.YarnClientImpl: Submitted application application_1610822007109_0006
21/01/16 20:20:20 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610822007109_0006/
21/01/16 20:20:20 INFO mapreduce.Job: Running job: job_1610822007109_0006
21/01/16 20:20:26 INFO mapreduce.Job: Job job_1610822007109_0006 running in uber mode : false
21/01/16 20:20:26 I

In [51]:
# parts b/c - take a look at the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -ls {HDFS_DIR}/vocabsize-output/

Found 4 items
-rw-r--r--   1 root supergroup          0 2021-01-16 20:20 /user/root/demo2/vocabsize-output/_SUCCESS
-rw-r--r--   1 root supergroup         20 2021-01-16 20:20 /user/root/demo2/vocabsize-output/part-00000
-rw-r--r--   1 root supergroup         20 2021-01-16 20:20 /user/root/demo2/vocabsize-output/part-00001
-rw-r--r--   1 root supergroup         19 2021-01-16 20:20 /user/root/demo2/vocabsize-output/part-00002


In [52]:
# parts b/c - view results (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/vocabsize-output/*


NumUniqueWords	1029
NumUniqueWords	1022
NumUniqueWords	970


> __DISCUSSION QUESTIONS (after breakout 3)__
* How many unique words were there?
* What happened when 3 reduce tasks were specified?

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (after breakout 3)__

*  How many unique words were there?
> There are 3021 unique words in this text. (NOTE: the count of unique words is dependent on the tokenizer, this count assumes the provided tokenizer was used.)  

*  What happened when 3 reduce tasks were specified?  
> Specifying three reduce tasks causes our result file to include 3 sub-counts instead of one total count. We'd need to add these three numbers to get the true total.

# Breakout 4: Secondary Sort
In breakout 1 we talked a little bit about the default sorting that Hadoop observes. However we'll often want to sort not just by the key but also by value. For example, we might want to sort the words by their count to find the most frequent words but then break ties by the word in alphabetical order. This is called a 'secondary sort'. In this question we'll learn about specifying parameters for sorting in Hadoop jobs. In particular you'll add three new parameters to your Hadoop Streaming command:  

__`-D stream.num.map.output.key.fields=2`__ : tells Hadoop to treat both the first and the second (tab separated) fields as a composite key.  

__`-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator`__ : tells Hadoop that we want to make comparisons (for sorting) based on the fields in this composite key

__`-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1"`__: Tells Hadoop to perform a reverse numerical sort on the second field in the composite key and then break ties by sorting (alphabetically) on the first field in the composite key.

To find the top words in the _Alice in Wonderland_ text we'll use the output of our word counting job as the input for this new sorting task. Recall that this output is a file in alphabetical order whose lines are of the format `word \t count`. Also recall that this file is already available in HDFS at the path `{HDFS_DIR}/wordcount-output`. You can simply pass this directory path in to the Hadoop streaming input parameter and it will understand that it should read in the directory's contents. __`IMPORTANT`:__ _please use a single reduce task for parts a and b._

__Documentation:__
* https://hadoop.apache.org/docs/r2.6.0/hadoop-mapreduce-client/hadoop-mapreduce-client-core/HadoopStreaming.html#Streaming_Command_Options   
* https://hadoop.apache.org/docs/r2.6.0/hadoop-mapreduce-client/hadoop-mapreduce-client-core/HadoopStreaming.html#Generic_Command_Options


> __DISCUSSION QUESTIONS (before breakout 4):__   
* Before we get to the full secondary sort it's worth noting that there is a really easy way to get Hadoop to sort our file by count: we could just switch the order of the count and the word when we print to standard output in our mapper. Why does this work?
* In the Hadoop job below we're using `/bin/cat` as our reducer... what is that?

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (before breakout 4)__
* Why would switching the order of the count & word achieve a secondary sort?
> This would work because Hadoop sorts on the key by default, so if the key is the count then we should get a list of words by frequency. 

* In the Hadoop job below we're using `/bin/cat` as our reducer... what is that?
 > remember that we can use any executable as a mapper or reducer. This is the bash command (binary excecutable) that just reads/passses whatever we give it.

### Breakout 4 Tasks:

* __a) code + discussion:__ Complete the code in __`TopWords/mapper.py`__ so that it performs the switch described above. For debugging purposes we'll first run this job using a test file of word counts instead of the full _Alice_ file. Run the provided Hadoop streaming command to confirm that our sneaky solution works. 
    * Notice that there is a problem with this result. Briefly discuss in groups what the problem is and why it happens.


* __b) code:__ Ok, for obvious reasons our 'sneaky' solution didn't quite give us the output we wanted. So let's do a secondary sort properly this time. To do this, add the three new Hadoop options described in the intro to this question. Run your job with these new specifications on the dummy count file. When you are satisfied that your job works, change the input path to specify the alice count output that is already in HDFS. Your list of top words should match the result you got in HW1.
    * __`Two important warnings:`__  1) Parameters starting with the `-D` flag must come immediately after the line where you specify the jar file and before the parameters `-files`, `-mapper`, etc; 2) The options we provided you above specify a reverse numeric sort on the second field and tie breaking using the first field... but the mapper you wrote in part a switched the order of the words and the counts. You will need to make a small adjustment to the options we provided so that it instead reverse numerically sorts by the first field and breaks ties on the second. 


* __c) code + discussion:__ Run your Hadoop job one more time but this time add the parameter to specify that the job should use 2 reduce tasks instead of 1. For convenience of illustration, you should do this using the sample counts file instead of the full _Alice_ text. 
    * Something is wrong with the results. Use the provided code to look at the output of each partition independently. Discuss why our results are off.  
    * Imagine we had a really large file and performed a sort using 100 partitions, what post processing would we have to do to get a fully ordered list (Total Order Sort)? Compare the computational cost of this postprocessing to the postprocessing we discussed in the VocabSize job.

__`NOTE:`__ The cell below will create a short file of word counts that we can load into HDFS and use to test our Hadoop MapReduce job. Take a moment to read this sample file and figure out what a reverse numerical sort (with alphabetical tie breaking) should yield. Then go on to complete your tasks as described above.

In [53]:
%%writefile TopWords/sample.txt
foo	5
quux	9
labs	100
bar	5
qi	1

Overwriting TopWords/sample.txt


In [54]:
# load sample file into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal TopWords/sample.txt {HDFS_DIR}

In [55]:
# part a - complete your work above then RUN THIS CELL AS IS
!chmod a+x TopWords/mapper.py

In [56]:
# parts a/b/c - clear output directory (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/topwords-output

rm: `/user/root/demo2/topwords-output': No such file or directory


In [57]:
# part a/b/c - Hadoop streaming command
!hadoop jar {JAR_FILE} \
  -files TopWords/mapper.py \
  -mapper mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/sample.txt \
  -output {HDFS_DIR}/topwords-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob8623500372000538363.jar tmpDir=null
21/01/16 20:20:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:20:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:21:00 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/16 20:21:00 INFO mapreduce.JobSubmitter: number of splits:2
21/01/16 20:21:00 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610822007109_0007
21/01/16 20:21:00 INFO impl.YarnClientImpl: Submitted application application_1610822007109_0007
21/01/16 20:21:00 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610822007109_0007/
21/01/16 20:21:00 INFO mapreduce.Job: Running job: job_1610822007109_0007
21/01/16 20:21:06 INFO mapreduce.Job: Job job_1610822007109_0007 running in uber mode : false
21/01/16 20:21:06 INFO mapreduce.Job:  map 0% reduce 0%
21/01

In [69]:
# <--- SOLUTION --->
# part a/b/c - Hadoop streaming command
!hdfs dfs -rm -r {HDFS_DIR}/topwords-output
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k1,1nr -k2,2" \
  -files TopWords/mapper.py \
  -mapper mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/sample.txt \
  -output {HDFS_DIR}/topwords-output \
  -numReduceTasks 3 \
  -cmdenv PATH={PATH}

Deleted /user/root/demo2/topwords-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob4298041091424481563.jar tmpDir=null
21/01/16 20:49:47 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:49:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:49:48 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/16 20:49:48 INFO mapreduce.JobSubmitter: number of splits:2
21/01/16 20:49:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610822007109_0009
21/01/16 20:49:49 INFO impl.YarnClientImpl: Submitted application application_1610822007109_0009
21/01/16 20:49:49 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610822007109_0009/
21/01/16 20:49:49 INFO mapreduce.Job: Running job: job_1610822007109_0009
21/01/16 20:49:55 INFO mapreduce.Job: Job job_1610822007109_0009 running in uber mode : false
21/01/16 20:49:55 IN

In [70]:
# part a/b/c - Save results locally (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/topwords-output/part-0000* > TopWords/results.txt

In [71]:
# part a/b/c - view results (RUN THIS CELL AS IS)
!head TopWords/results.txt

5	bar	
5	foo	
100	labs	
9	quux	
1	qi	


In [72]:
# part c - look at first partition (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/topwords-output/part-00000

5	bar	
5	foo	


In [73]:
# part c - look at second partition (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/topwords-output/part-00001

100	labs	
9	quux	
1	qi	


__Expected Results:__

<table>
<th>part a</th>
<th>part b</th>
<th>part c</th>
<tr>
<td><pre>
1	qi
100	labs
5	foo
5	bar
9	quux
</pre></td>
<td><pre>
100	labs
9	quux
5	bar
5	foo
1	qi
</pre></td>
<td><pre>
9	quux
5	bar
100	labs
5	foo
1	qi
</pre></td>
</tr></table>

>__DISCUSSION QUESTIONS__:
* What was the problem with the results in part a?
* How do you know the secondary sort worked in part b?
* What was the problem with the results in part c?
* How could we post process these files (part c partitions) to produce a total order sort of them? Why would this be computationally expensive?

### <--- SOLUTION --->
__INSTRUCTOR TALKING POINTS (after breakout 4)__

* What was the problem with the results in part a?
> There is a big and a little problem here. The little problem is that we've sorted from smallest to largest when we'd probably have the opposite order if we're looking for a list of most frequent words. The bigger problem is that 'labs 100' isn't ordered correctly. It should be last but instead its second. This occurs because Hadoop is treating the counts as text values not as a numbers so they're sorted alphabetically not by value.

* How do you know the secondary sort worked in part b?
> there are two records with the same count (5) and the 'bar' comes before the 'foo'

* What was wrong with the results in part c?
> The results are not sorted properly because when we specify 2 reducers we end up with two sorted files concatenated together.  

* How could we post-process these files to produce a total order sort of them? Why would this be computationally expensive?
> To get a single list of words by frequency we'd need to merge sort these files (partitions) together. This is a much more computationally challenging task than simply adding the result of each mapper. In particular if we had lots of partitions (eg. 100) this mergesort would be O(100∗|V|).

# Breakout 5: Tracking Down Errors in Python Code
You've now seen most of the basic functionality of writing and running Hadoop streaming jobs. In this week's homework and over the course of the next few weeks we'll explore additional options and tricks to add to our jobs. As we do this you will want to be able to quickly distinguish between errors that occur due to a mistake in your algorithm design or Hadoop streaming command and errors that are rooted in your Python code. Unfortunately the error logs printed to console do not always make this distinction obvious. Luckily, the Hadoop UI logs do make it very easy to identify Python coding errors. Before you move on to the homework (even if there isn't time in class) we'd like to make sure you know where to find these logs and how to fix two common mistakes. __Below, we provided code that contains two common errors for you to debug. Your job is to:__
1. __Run the provided code as is, it will throw an error.__

2. __Navigate to the Hadoop UI and find the relevant logs explaining your error.__ 
 * Under `Task Type`, click `Map` > `task_XXXXXX` >`logs`

3. __Fix the error(s) and re-run the job__. 

__`NOTE 1:`__ There are two different kinds of errors in the mapper code. See the inline comments for specific fixes: one involves adding a parameter to your Hadoop job the other two you must fix in the mapper code (re-run that cell to overwrite the old mapper). I'd recommend fixing them one at a time so that you can see how the logs and error messages change depending on the type of error.

__`NOTE 2:` If you do not get to this section in class, you still must complete it before beginning the homework!__

In [74]:
%%writefile TopWords/sample.txt
foo	5
quux	9
labs	100
bar	5
qi	1

Overwriting TopWords/sample.txt


In [75]:
# run the following cells to create the demo mapper
!mkdir demo

mkdir: cannot create directory `demo': File exists


The next cell uses a little Jupyter Magic to create a python script on the fly, this is a useful technique that you may want to use in the future to create files for unit testing, etc.

In [76]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This is a silly mapper to demonstrate some errors.
"""
import sys
import numpy as np  # To use numpy add -cmdenv PATH={PATH} to your Hadoop Job

for line in sys.stdin:
    msg = ("a message"   # missing a parenthesis here
    print(1/0)            # dividing by zero is a no-go

Overwriting demo/mapper.py


In [77]:
# clear HDFS output directory when you re-run the job
!hdfs dfs -rm -r {HDFS_DIR}/demo-output

rm: `/user/root/demo2/demo-output': No such file or directory


In [78]:
# Hadoop streaming command
!hdfs dfs -rm -r {HDFS_DIR}/demo-output
!hadoop jar {JAR_FILE} \
  -files demo/mapper.py \
  -mapper mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/sample.txt \
  -output {HDFS_DIR}/demo-output \
  -cmdenv PATH={PATH}

rm: `/user/root/demo2/demo-output': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob9580416457511258.jar tmpDir=null
21/01/16 20:53:55 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:53:56 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:53:56 INFO mapred.FileInputFormat: Total input paths to process : 1
21/01/16 20:53:56 INFO mapreduce.JobSubmitter: number of splits:2
21/01/16 20:53:56 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1610822007109_0010
21/01/16 20:53:56 INFO impl.YarnClientImpl: Submitted application application_1610822007109_0010
21/01/16 20:53:57 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1610822007109_0010/
21/01/16 20:53:57 INFO mapreduce.Job: Running job: job_1610822007109_0010
21/01/16 20:54:03 INFO mapreduce.Job: Job job_1610822007109_0010 running in uber mode : false
21

In [79]:
# <--- SOLUTION --->
# Hadoop streaming command
!hadoop jar {JAR_FILE} \
  -files demo/mapper.py \
  -mapper mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/sample.txt \
  -output {HDFS_DIR}/demo-output \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.16.2.jar] /tmp/streamjob2110567459188237115.jar tmpDir=null
21/01/16 20:54:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:54:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/01/16 20:54:23 WARN security.UserGroupInformation: PriviledgedActionException as:root (auth:SIMPLE) cause:org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory hdfs://quickstart.cloudera:8020/user/root/demo2/demo-output already exists
21/01/16 20:54:23 WARN security.UserGroupInformation: PriviledgedActionException as:root (auth:SIMPLE) cause:org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory hdfs://quickstart.cloudera:8020/user/root/demo2/demo-output already exists
21/01/16 20:54:23 ERROR streaming.StreamJob: Error Launching job : Output directory hdfs://quickstart.cloudera:8020/user/root/demo2/demo-output already exists
Streaming Command Failed!


### Follow-Up:  
* The below images should be similar to what was found during the above exercise. Before beginning homework 2, make sure you can find these errors through the Hadoop UI. 
    * If you are unable to find these errors in the Hadoop UI logs, **seek the help of an instructor or TA immediately**.

![hadoop-module-error](../failed-job-01.png)
![hadoop-module-error](../failed-job-02.png)
![hadoop-module-error](../failed-job-03.png)
![hadoop-module-error](../failed-job-04.png)